In [3]:
from xml.dom.minidom import parse
import xml.dom.minidom
import latlng
import math

DOMTree = xml.dom.minidom.parse("maps/manhattan.xml")
collection = DOMTree.documentElement

nodes = collection.getElementsByTagName("node")
ways = collection.getElementsByTagName("way")


In [4]:
nodes_ll = {}
print(len(nodes))
for node in nodes:
    nodes_ll[node.getAttribute("id")] = (float(node.getAttribute("lat")),float(node.getAttribute("lon")))

1101031


In [6]:
print(len(ways))

177829


In [5]:
def get_latlng(ref):
    return nodes_ll[ref]

class node:
    
    def __init__(self, nd):
        self.nd = nd
        self.ref = nd.getAttribute("ref")
        ll = get_latlng(self.ref)
        self.lat = ll[0]
        self.lng = ll[1]

class road:
    
    def __init__(self, way, category):
        self.way = way
        self.category = category
        self.nodes = []
        for nd in way.getElementsByTagName("nd"):
            self.nodes.append(node(nd))
    
    def get_distance(self, lat, lng):
        l = len(self.nodes)
        d = 0
        mdis = 99999999999
        md = 0
        for i in range(1,l):
            p1 = latlng.Cartesian(0,0)
            p2 = latlng.LatLng(lat,lng).get_xy(latlng.LatLng(self.nodes[i-1].lat, self.nodes[i-1].lng))
            p3 = latlng.LatLng(lat,lng).get_xy(latlng.LatLng(self.nodes[i].lat, self.nodes[i].lng))
            k = ( (p1.x - p2.x)*(p3.x - p2.x) + (p1.y - p2.y)*(p3.y - p2.y) ) / \
                ( (p3.x - p2.x)**2 + (p3.y - p2.y)**2 )
            foot = latlng.Cartesian(p2.x + k*(p3.x - p2.x), p2.y + k*(p3.y - p2.y))
            if p2.x <= foot.x and foot.x <= p3.x and p2.y <= foot.y and foot.y <= p3.y:
                d += math.sqrt( (p2.x-foot.x)**2 + (p2.y-foot.y)**2 )
                dis = math.sqrt( foot.x**2 + foot.y**2 )
                if dis < mdis:
                    mdis = dis
                    md = d
            else:
                d += math.sqrt( (p2.x-p3.x)**2 + (p2.y-p3.y)**2 )
        return (mdis, md)
        

In [7]:
import matplotlib.pyplot as plt

roads = []

for way in ways:
    ndx = []
    ndy = []
    for tag in way.getElementsByTagName("tag"):
        if tag.getAttribute("k") == "highway"\
        and (tag.getAttribute("v")=="motorway"\
        or tag.getAttribute("v")=="trunk"\
        or tag.getAttribute("v")=="primary"\
        or tag.getAttribute("v")=="secondary"\
        or tag.getAttribute("v")=="tertiary"\
        or tag.getAttribute("v")=="unclassified"\
        or tag.getAttribute("v")=="residential"):
            r = road(way, tag.getAttribute("v"))
            l = len(r.nodes)
            #for i in range(l):
            #    ndx.append(r.nodes[i].lng)
            #    ndy.append(r.nodes[i].lat)
            #plt.plot(ndx, ndy)
            roads.append(r)
            break
#plt.show()

In [13]:
import treefinder
tf = treefinder.TreeFinder("trees_man.csv")


In [14]:
roads_new = []
for r in roads:
    t = False
    for p in r.nodes:
        tree = tf.find_the_nearest_tree(p.lat, p.lng, 25)
        if tree == None:
            continue
        else:
            t = True
            break
    if t:
        roads_new.append(r)
roads = roads_new

In [16]:
doc = xml.dom.minidom.Document()
root = doc.createElement('roads')
doc.appendChild(root)
for r in roads:
    road = doc.createElement('road')
    road.setAttribute('category',r.category)
    for node in r.nodes:
        nd = doc.createElement('node')
        nd.setAttribute('ref',str(node.ref))
        nd.setAttribute('lat',str(node.lat))
        nd.setAttribute('lng',str(node.lng))
        road.appendChild(nd)
    root.appendChild(road)
f = open('road_network_man.xml', 'w')
doc.writexml(f, indent='\t', addindent='\t', newl='\n', encoding="utf-8")
f.close()